In [1]:
import cv2
import numpy as np

def correct_frames_with_optical_flow(video_path):
    cap = cv2.VideoCapture(video_path)

    # Get the width and height of the frames
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Lucas-Kanade parameters
    lk_params = dict(winSize=(15, 15), criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

    # Read the first frame
    ret, prev_frame = cap.read()

    if not ret:
        print("Error: Couldn't read the first frame.")
        return

    # Convert the first frame to grayscale
    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

    # Detect good features to track in the first frame
    prev_pts = cv2.goodFeaturesToTrack(prev_gray, maxCorners=500, qualityLevel=0.01, minDistance=10)

    if prev_pts is None:
        print("Error: No features detected in the first frame.")
        return

    while True:
        ret, frame = cap.read()

        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Calculate optical flow using Lucas-Kanade method
        flow = cv2.calcOpticalFlowPyrLK(prev_gray, gray, prev_pts, None, **lk_params)

        # Extract the new position of features
        good_new = flow[0].reshape(-1, 2)

        # Select good points
        mask = flow[1].reshape(-1) == 1
        good_pts = good_new[mask]

        # Check if the number of points is the same
        if len(good_pts) != len(prev_pts):
            print("Error: Number of points changed. Skipping frame.")
            continue

        # Calculate the displacement vector for each point
        displacement = good_pts - prev_pts.reshape(-1, 2)

        # Apply the displacement vector to the original image
        corrected_frame = np.zeros_like(frame)
        corrected_frame[:, :] = frame[:, :]  # Copy original frame

        # Apply the displacement vector to each pixel in the frame
        for i, (x, y) in enumerate(good_pts):
            x, y = int(x), int(y)
            dx, dy = displacement[i].astype(int)

            if 0 <= x + dx < width and 0 <= y + dy < height:
                corrected_frame[y, x] = frame[y + dy, x + dx]

        # Display the corrected frame
        cv2.imshow('Corrected Frame', corrected_frame)

        # Update previous frame and points
        prev_gray = gray
        prev_pts = good_pts  # Update with the new points

        # Break the loop if 'q' key is pressed
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Replace 'your_video_path.mp4' with the actual path to your video file
video_path = 'fullstream.mp4'
correct_frames_with_optical_flow(video_path)


Error: Number of points changed. Skipping frame.
Error: Number of points changed. Skipping frame.
Error: Number of points changed. Skipping frame.
Error: Number of points changed. Skipping frame.
Error: Number of points changed. Skipping frame.
Error: Number of points changed. Skipping frame.
Error: Number of points changed. Skipping frame.
Error: Number of points changed. Skipping frame.
Error: Number of points changed. Skipping frame.
Error: Number of points changed. Skipping frame.
Error: Number of points changed. Skipping frame.
Error: Number of points changed. Skipping frame.
Error: Number of points changed. Skipping frame.
Error: Number of points changed. Skipping frame.
Error: Number of points changed. Skipping frame.
Error: Number of points changed. Skipping frame.
Error: Number of points changed. Skipping frame.
Error: Number of points changed. Skipping frame.
Error: Number of points changed. Skipping frame.
Error: Number of points changed. Skipping frame.
Error: Number of poi

In [4]:
import cv2 as cv 
import numpy as np 


# The video feed is read in as 
# a VideoCapture object 
cap = cv.VideoCapture("fullstream.mp4") 

# ret = a boolean return value from 
# getting the frame, first_frame = the 
# first frame in the entire video sequence 
ret, first_frame = cap.read() 

# Converts frame to grayscale because we 
# only need the luminance channel for 
# detecting edges - less computationally 
# expensive 
prev_gray = cv.cvtColor(first_frame, cv.COLOR_BGR2GRAY) 

# Creates an image filled with zero 
# intensities with the same dimensions 
# as the frame 
mask = np.zeros_like(first_frame) 

# Sets image saturation to maximum 
mask[..., 1] = 255

while(cap.isOpened()): 
	
	# ret = a boolean return value from getting 
	# the frame, frame = the current frame being 
	# projected in the video 
	ret, frame = cap.read() 
	
	# Opens a new window and displays the input 
	# frame 
	cv.imshow("input", frame) 
	
	# Converts each frame to grayscale - we previously 
	# only converted the first frame to grayscale 
	gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY) 
	
	# Calculates dense optical flow by Farneback method 
	flow = cv.calcOpticalFlowFarneback(prev_gray, gray, 
									None, 
									0.5, 3, 15, 3, 5, 1.2, 0) 
	
	# Computes the magnitude and angle of the 2D vectors 
	magnitude, angle = cv.cartToPolar(flow[..., 0], flow[..., 1]) 
	
	# Sets image hue according to the optical flow 
	# direction 
	mask[..., 0] = angle * 180 / np.pi / 2
	
	# Sets image value according to the optical flow 
	# magnitude (normalized) 
	mask[..., 2] = cv.normalize(magnitude, None, 0, 255, cv.NORM_MINMAX) 
	
	# Converts HSV to RGB (BGR) color representation 
	rgb = cv.cvtColor(mask, cv.COLOR_HSV2BGR) 
	
	# Opens a new window and displays the output frame 
	cv.imshow("dense optical flow", rgb) 
	
	# Updates previous frame 
	prev_gray = gray 
	
	# Frames are read by intervals of 1 millisecond. The 
	# programs breaks out of the while loop when the 
	# user presses the 'q' key 
	if cv.waitKey(1) & 0xFF == ord('q'): 
		break

# The following frees up resources and 
# closes all windows 
cap.release() 
cv.destroyAllWindows() 
